In [1]:
import os
import folium
import json
#import geopandas
import pandas as pd
import numpy as np

In [2]:
china = os.path.join('china_provinces_en.json')
geo_json_data = json.load(open(china))

In [3]:
m = folium.Map([41,100],zoom_start = 3.2)
folium.GeoJson(geo_json_data).add_to(m)
m.save(os.path.join('GeoJSON_and_choropleth_0.html'))
m

In [4]:
ag_data = pd.read_csv('Agricultural total production.csv')

In [5]:
ag_data.head()

,city,city_num,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,bei_jing,1.0,145.20,154.48,155.10,170.41,166.29,163.37,154.2,140.44,128.10,115.48
1,tian_jin,2.0,244.31,238.02,230.74,217.16,195.99,179.87,168.3,139.70,127.67,117.60
2,he_bei,3.0,3459.39,3441.37,3453.42,3473.27,3095.29,2775.27,2470.1,1927.78,1760.75,1639.07
3,shan_xi_2,4.0,958.11,969.52,984.03,932.14,847.41,767.14,669.0,556.34,366.16,322.65
4,nei_meng_gu,5.0,1415.07,1418.31,1408.44,1328.07,1171.97,1057.85,900.4,731.90,716.61,620.42


In [ ]:
Geo = pd.read_csv('LocationGeo.csv')

In [6]:
from branca.colormap import linear
colormap = linear.YlGn.scale(ag_data['2016'].min(),ag_data['2016'].max())#color:Y1Gn
print(colormap(5.0))
colormap

#ffffcc


In [7]:
colormap1 = linear.PuBu.scale(Geo['res_2016'].min(),Geo['res_2016'].max())
print(colormap1(5.0))
colormap1

NameError: name 'Geo' is not defined

In [ ]:
ag_dict = ag_data.set_index('city')['2016']
ag_dict['bei_jing']

In [ ]:
Geo['2016'] = Geo['2016'].astype(str)

In [ ]:
locations = Geo[['longitude','latitude']]
locationlist = locations.values.tolist()

In [ ]:
res_dict = Geo.set_index('city')['res_2016']

In [ ]:
Geo['res_2016'] = Geo['res_2016'].astype(str)

In [ ]:
m = folium.Map([41,100], zoom_start = 4)
for point in range(0,len(locationlist)):
    folium.Marker(location = locationlist[point], popup = str('The annual agricultural production in '+Geo['formal_city'][point]+' is '+Geo['2016'][point]+'\n The restaurant amount in '+Geo['formal_city'][point]+' is '+Geo['res_2016'][point])).add_to(m)
folium.GeoJson(geo_json_data, name = 'agricultural_production', 
               style_function = lambda feature:{'fillColor':colormap(ag_dict[feature['id']]),
                                                'color':'black','weight':1,'dashArray':'5,5','fillOpacity':0.5}).add_to(m)
folium.GeoJson(geo_json_data, name = 'restaurant_amount',
             style_function = lambda feature:{'fillColor':colormap1(res_dict[feature['id']]),
                                             'color':'purple','weighth':1,'dashArray':'5.5','fillOpacity':0.5}).add_to(m)
folium.LayerControl().add_to(m)
m.save(os.path.join('GeoJSON_and_choropleth_5.html'))
m